# Языковые модели

Какое слово в последовательности вероятнее: 

Поезд прибыл на
* вокзал
* север

Какая последовательность вероятнее:
* Вокзал прибыл поезд на
* Поезд прибыл на вокзал

Языковая модель [language model, LM]  позволяет оценить вероятность следующего слова в последовательности  $P(w_n | w_1, \ldots, w_{n-1})$ и оценить вероятность всей последовательности слов $P(w_1, \ldots, w_n)$.

## Модель $n$-грам

Пусть $w_{1:n}=w_1,\ldots,w_m$ – последовательность слов.

Цепное правило:
$ P(w_{1:m}) = P(w_1) P(w_2 | w_1) P(w_3 | w_{1:2}) \ldots P(w_m | w_{1:m-1}) = \prod_{k=1}^{m} P(w_k | w_{1:k-1}) $

Но оценить $P(w_k | w_{1:k-1})$ не легче! 

Переходим к $n$-грамам: $P(w_{i+1} | w_{1:i}) \approx P(w_{i+1} | w_{i-n:i})  $ , то есть, учитываем $n-1$ предыдущее слово. 

Модель
* униграм: $P(w_k)$
* биграм: $P(w_k | w_{k-1})$
* триграм: $P(w_k | w_{k-1} w_{k-2})$


Т.е. используем Марковские допущения о длине запоминаемой цепочки.

* Вероятность следующего слова в последовательности: $ P(w_{i+1} | w_{1:i}) \approx P(w_{i-n:i}) $
* Вероятность всей последовательности слов: $P(w_{1:n}) = \prod_{k=1}^l P(w_k | w_{k-n+1: k-1}) $

### Качество модели  $n$-грам

Перплексия: насколько хорошо модель предсказывает выборку. Чем ниже значение перплексии, тем лучше.

$PP(\texttt{LM}) = b^{{-{\frac  {1}{N}}\sum _{{i=1}}^{N}\log _{b}\texttt{LM}(x_{i})}}$,

$N$ — длина корпуса<br>$x_i$ — i-тое слово в корпусе<br>LM(x) — предсказание вероятности языковой моделью<br>b — некоторая константа, обычно 2

## Счетные языковые модели

### ММП оценки вероятностей
$ P_{MLE}(w_k | w_{k-n+1:k-1}) = \frac{\texttt{count}(w_{k-n+1:k-1} w_k )}{\texttt{count}(w_{k-n+1:k-1} )} $

В модели биграм:

$ P_{MLE}(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k )}{\texttt{count}(w_{k-1} )} $

### Аддитивное сглаживание Лапласа

$ P(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k ) + \alpha}{\texttt{count}(w_{k-1} ) + \alpha |V|} $

$|V|$ -- размер словаря

## Модели биграм в NLTK

In [1]:
import nltk

In [2]:
names = [name.strip().lower() for name in open('dinos.txt').readlines()]
print(names[:10])

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor']


In [3]:
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

print(list(freq.keys()))

['a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'g', 'f', 'j', 'k', 'w', 'z', 'q']


In [4]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
print(cfreq['a'])

<FreqDist with 26 samples and 2487 outcomes>


In [5]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0105
p(a b) = 0.0129
p(a u) = 0.3185


In [6]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1354


In [7]:
# можно порождать случайные символы с учётом предыдущих 
cprob['a'].generate()

's'

### Задание 1

Напишите функцию для генерации нового имени динозавра фиксированной длины.

## Рекуррентные нейронные языковые модели

RNN позволяют уйти от Марковских допущений и позволяют учитывать предысторию произвольной длины.

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

$y_n = RNN(x_{1:n})$, $y_n \in \mathbb{R}^{d_{out}}$

Для каждого префикса $x_{i:i}$ $y_i$ – выходной вектор.

$y_i = RNN(x_{1:i})$

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

$R$ –  рекурсивная функция с двумя входами: $x_i$ и $s_{i-1}$ (вектор состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(s_{i-1}* W^s + x_i W^x +b)$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{d_{out}}$, $s_i \in \mathbb{R}^{d_{out}}$

$W^x \in \mathbb{R}^{d_{in} \times d_{in}}$, $W^s \in \mathbb{R}^{d_{out} \times d_{out}}$

![rnn](images/rnn.png)

In [9]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

In [11]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content))
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        #teacher forcing
        x_str = line
        y_str = line[1:] + '\n'
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [12]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [13]:
print(trn_ds.lines[1])

aardonyx


In [14]:
print(trn_ds.ch_to_idx)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [15]:
x, y = trn_ds[1]
print(x)
print(y)

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])
tensor([ 1, 18,  4, 15, 14, 25, 24,  0])


![rnn](images/dinos3.png)

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) # конкатенируем вектора состояния и входа
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(combined)
        return h, y

In [17]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [18]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end='') for x in sample_idxs]

In [19]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['\n']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = random.randint(1, trn_ds.vocab_size-1)
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != newline_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

In [20]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()

In [21]:
def train(model, loss_fn, optimizer, dataset='dinos', epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)
        print()

In [22]:
train(model, loss_fn, optimizer, epochs = 50)

Epoch:1
dcastiosx
yc
kxbretacmosaru
dsrlerlh
yualuotaourus
usnwnurus
yjmaioerus
ncrnaneour
buaktdurus
vlipsaurus
puainauris
fuadsaarus
mgasbaurus
ztsigaurus
gprtaaueus

Epoch:2
auruh
zrnaoauruo
canutrhsnurds
riytoiur
zinaigerus
odonapaour
yucitaurus
sanrusuurus
yamialhkrus
pariaarrus
sodruspurus
netopvurus
iamiamoaops
targaibs
hebsaoturus

Epoch:3
drneosaurus
trbgnces
lpltcaur
jigrinaurus
guersuresausus
nptahocaurus
paernasaurus
jamvosaurus
jqriysaurui
ticcsasauhus
yanesauour
kwntcsiurus
riytorouakraurus
yubdsadrus
iebransurus

Epoch:4
erilopnuurus
xainaisaurus
wrnaoihurus
nixoucsmurus
antuscuras
obataopsahor
wiioinaurus
quatssaurus
osgptocapels
loltadoudus
xihbsarpurus
oshitorytnosaarus
zrcpisarrue
dergicrcaurus
auroraurus

Epoch:5
notocauhus
rtmtamopaor
vematarmurus
loneosaurus
lubgocaunus
madtoaaurus
quanusptrus
marrruurus
kamiahsaurus
lopaoemuras
xivntdlurus
gplysnsau
anrarlerus
vdriaonaurus
antosaurus

Epoch:6
orontor
notamraurus
srcrtraurus
aararturos
lqngorgysaurur
bicgrciuris
y

veragor
ypasjlaurus
euaursaurus
yuiposaurus
gargtdasa
apcur
tasantkras
usnlhunuusaus
sancosaurus
ytibhethp
brcivaurus
jianijvaurus
fnlachasaurus
napgrnaurus
chylosaurus

Epoch:45
tytirou
fln
zhubheucdsacrus
qoanblpurus
trpastosnitru
rhnaisaston
osnateone
mpcitaurus
dolgtlvourus
ynebisaurus
brmangtsatrus
xixgosiurus
tytiyroallceeas
lubeoraurus
varanyssurus

Epoch:46
teponturus
kaleanilrus
ceraaoneurus
onusturus
xinlkztaurus
wedesesaurus
falesaodus
lupsespsths
xixoinos
urtaaucus
zaash
ypavosaurus
juassosaurus
wentobisaurus
xqadoraurus

Epoch:47
qarbksaurus
biiotitoptor
thranocoro
parlanaurus
waltntasturus
hiyomcuros
zainasitmrus
zheqgkdhurus
partrmctoryx
viptcopaat
grltanosaurus
fasarustirax
sapthuugouagrus
ructosaurus
fesgraudus

Epoch:48
ptktysaurus
osiktoaniilormit
kasibrhs
licoantosaurus
fusmtturus
khnamocaurus
ribpisaurus
ptrugostiryx
gyrnosorrus
yuaugsairus
waniicn
zhunyesorsaurus
pirtrbaurus
erctaiopaor
phlaraisosausus

Epoch:49
drscxoraurps
zainaono
lasibepitops
chuitispurus
liwi

### Задание 2
Измените код выше так, чтобы генерировались панграмы – имена динозавров, не содержащие повторяющихся букв

## Использование LSTM нейронов

![rnn](images/LSTM_rnn.png)

Рассмотрим один блок поближе:

![lstm](images/understanding_lstms.jpg)

In [23]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.linear_f = nn.Linear(input_size + hidden_size, hidden_size)
        self.linear_u = nn.Linear(input_size + hidden_size, hidden_size)
        self.linear_c = nn.Linear(input_size + hidden_size, hidden_size)
        self.linear_o = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.i2o = nn.Linear(hidden_size, output_size)
        
    def forward(self, c_prev, h_prev, x):
        combined = torch.cat([x, h_prev], 1)
        f = torch.sigmoid(self.linear_f(combined))
        u = torch.sigmoid(self.linear_u(combined))
        c_tilde = torch.tanh(self.linear_c(combined))
        c = f*c_prev + u*c_tilde
        o = torch.sigmoid(self.linear_o(combined))
        h = o*torch.tanh(c)
        y = self.i2o(h)
        
        return c, h, y

In [24]:
model = LSTM(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [25]:
def sample(model):
    model.eval()
    with torch.no_grad():
        c_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        h_prev = torch.zeros_like(c_prev)
        idx = random.randint(1, 26)
        x = c_prev.new_zeros([1, trn_ds.vocab_size])
        x[0, idx] = 1
        sampled_indexes = [idx]
        n_chars = 1
        newline_char_idx = trn_ds.ch_to_idx['\n']
        while n_chars != 50 and idx != newline_char_idx:
            c_prev, h_prev, y_pred = model(c_prev, h_prev, x)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=torch.softmax(y_pred, 1).cpu().numpy().ravel())
            sampled_indexes.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
            
            n_chars += 1
            
            if n_chars == 50:
                sampled_indexes.append(newline_char_idx)
                
    model.train()
    return sampled_indexes

In [26]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        c_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        h_prev = torch.zeros_like(c_prev)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            c_prev, h_prev, y_pred = model(c_prev, h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()

In [27]:
train(model, loss_fn, optimizer, epochs = 50)

Epoch:1
wimaitcois
daslakgsngasgshusus
prngosgytesaurus
yiobqit
lariapaurus
wamvotaurus
jrpnysaurus
chbatar
itbcrraurus
nataitaurus
zhkosaurus
btamlcaurus
ocdqraurus
nasbpturus
vinatopuus

Epoch:2
ostbipaurus
nuberacops
ulatanusaurus
waponysaurus
vecaters
jaapk
medsknaurus
itcstsaurus
kunostnrus
xiangucauros
yuhicaurus
hassssaurus
terpsaurus
taretadaurus
ihg

Epoch:3
trcitburus
licnootaurus
ornaioasaurus
iandosaurus
kotcusaurus
syurortaraurus
qiubashcrus
recpanuurus
vincssurus
brsanodaurus
samrobaurus
tociuaurus
unbskurus
jonbgodaurus
pachoacurus

Epoch:4
stantaurus
orhithusaurus
foaesaurus
ropaodronataran
kasssasrunuurus
palecosaurus
stlangolsaurus
otcosaurus
gwycous
yinaercsaurus
handungasaurus
prrongsaurus
etsanmacsaurus
asgaeerpaurus
coycrsoptor

Epoch:5
qunntosasaurus
ketaetobaurus
juagysaurus
senorturus
daegeiseudaurus
anbtisaurus
luattsaurus
lunltsaurus
acreucaurus
prlccocaurus
istnisaurus
misahicosaurus
gonaolosaurus
urtasossaurus
wanstasras

Epoch:6
hargtcauras
yuihasaurus
qul

kantrydithoiosrsr
arptaeocaurus
paciedsaurus
yantullosaurus
dytgpsaurus
euconilta
canatosaurus
zuengjinaurus
miruaraurus
gijantocaurus
xuanzanisaurus
chyogruoglaurus
nesabinaosaurus

Epoch:43
nevoyansosaurus
riptodaurus
wilvemrnasaurus
udhuksgosaurus
kutiasocaurus
nootaurus
vardodaurus
agphaeorniuhomimos
daltaiaaurus
pilsliaropaurus
isdesaurus
istaisaurus
fuhiis
konmanas
jiugengssaurus

Epoch:44
tonosscaurus
grlvensaurus
gibveotaurus
fuksoruurus
xaen
ceuhghocaurus
quilasaurus
wonkersaurus
futaisaurus
juafonfosaurus
tevmtlaurus
brmvsaurus
klceakosaurus
bigpiias
hirrosturus

Epoch:45
drgyoosourus
diycopuliti
walseoaomaurus
trtinpurus
hunraraurus
heraosaurus
tecocaurus
lorilor
evsaurus
necdhepsaurus
andropaus
wylwerrurus
sewropstas
haaxasetaas
cainshsaurus

Epoch:46
kumonsosaurus
lintaracaurus
gaganatis
yoctaouos
lonsenoosaurus
uainahuanoaurus
thesraurus
zhungornpaurus
fusuiraurus
ultidaurus
frlerbaurus
brreersaurus
duagmoanaauras
xiaidosaurus
ortithosaurus

Epoch:47
walnulsaurus
deycaeoc

### Задание 3
Написать функцию ```get_prob()```, оценивающую веростность порождения одной строки (из файла) и найти самую вероятную строку, порождаемую каждой из трех языковых моделей.

### Задание 4
Используя функцию ```get_prob()```, написать функцию ```perplexety ```, оценивающую перплексию каждой из трех языковых моделей.

### Задание 5
Добавить в функцию сэмплирования возможность использовать т.н. температуру. Сейчас сэмплирование следующего символа осуществляется с использованием ```np.random.choice```, где вероятность каждого символа получена на выходе из ```softmax```:

```p=torch.softmax(y_pred, 1).cpu().numpy().ravel()```.


Температура сэмплирования определяется параметром $T$ и преобразует вероятности следующим образом: $p_i = \frac{p_i^{1/T}}{\sum_j p_j^{1/T}}$ .

Проведите эксперименты с разными значениями $T \in [0.5, 1, 2]$. Как разные значения $T$ влияют на генерируемые строки?

### Задание 6
Реализуйте beam search для генерации строк. 

# Ссылки

1. Сэмплирование в  RNN: https://nlp.stanford.edu/blog/maximum-likelihood-decoding-with-rnns-the-good-the-bad-and-the-ugly/
2. Основной источник 1: https://github.com/furkanu/deeplearning.ai-pytorch/tree/master/5-%20Sequence%20Models
3. Основной источник 2: https://github.com/Kulbear/deep-learning-coursera/blob/master/Sequence%20Models/Dinosaurus%20Island%20--%20Character%20level%20language%20model%20final%20-%20v3.ipynb
4. LSTM: http://colah.github.io/posts/2015-08-Understanding-LSTMs/